# Evaluation Notebook for CNAE Group 11 Experiments

### How to:
#### Use the next code block to upload Locust datasets (or mount your Google Drive) to add data.


1.   exceptions.csv
2.   failures.csv
3.   report.html
4.   requests.csv


Upload files

In [1]:
# from google.colab import files
# uploaded = files.upload()

ModuleNotFoundError: No module named 'google'

Connect Drive

In [ ]:
#or connect you drive to add the csv-files
#from google.colab import drive
#drive.mount('/content/drive')

Reference locally

In [ ]:
exceptions = Data/exceptions.csv
failures = Data/failures.csv
requests = Data/requests.csv

NameError: ignored

In [ ]:
#initial installation

!pip install matplotlib
!pip install math

ERROR: Could not find a version that satisfies the requirement math (from versions: none)
ERROR: No matching distribution found for math


In [ ]:
# install packages
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

# Histograms
### Failures

In [ ]:
p, edges = np.histogram(failures.csv)